<a href="https://colab.research.google.com/github/SamarSaeed1146/Internet_Chatbot/blob/main/Internet_Chatbot_With_Short_Memory_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>